In [45]:
# NB13 — Feature Refresh + Ablation + Prod Comparison
import os, json, math, numpy as np, pandas as pd
from pathlib import Path
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    roc_auc_score, average_precision_score, brier_score_loss, log_loss
)
import warnings
warnings.filterwarnings("ignore")

SEED = 42
np.random.seed(SEED)

DATA_DIR = Path("data")
ART_DIR  = Path("artifacts")
FIG_DIR  = Path("reports/figures")
for p in [DATA_DIR, ART_DIR, FIG_DIR]: p.mkdir(parents=True, exist_ok=True)

# Load df (parquet preferred)
df_path = DATA_DIR/"df_nb02.parquet" if (DATA_DIR/"df_nb02.parquet").exists() else DATA_DIR/"df_nb02.csv"
df = pd.read_parquet(df_path) if df_path.suffix==".parquet" else pd.read_csv(df_path, parse_dates=["date"])

# Normalize date & sort
df["date"] = pd.to_datetime(df["date"], errors="coerce")
df = df.dropna(subset=["date"]).sort_values("date").reset_index(drop=True)

# Ensure label 'y' exists (next-day up)
if "y" not in df.columns:
    if "ret1" in df.columns:
        df["y"] = (df["ret1"].shift(-1) > 0).astype(int)
    elif "close" in df.columns:
        df["y"] = (df["close"].pct_change().shift(-1) > 0).astype(int)
    else:
        raise KeyError("No label y, ret1, or close available to derive labels.")

print(f"Rows/Features: {len(df)} / {df.shape[1]}")
print(f"Span: {df['date'].min().date()} → {df['date'].max().date()}")
print("Pos rate:", round(float(df["y"].mean()), 3))


Rows/Features: 2686 / 22
Span: 2015-02-06 → 2025-10-10
Pos rate: 0.53


In [46]:
def metric_block(y_true, y_prob):
    p = np.clip(np.asarray(y_prob), 1e-15, 1 - 1e-15)
    return {
        "AUC": roc_auc_score(y_true, p),
        "PR_AUC": average_precision_score(y_true, p),
        "Brier": brier_score_loss(y_true, p),
        "LogLoss": log_loss(y_true, p),
    }

def sincos_time(dates, period=7):
    dt = pd.to_datetime(dates, errors="coerce")
    dtd = dt.dt if hasattr(dt, "dt") else dt
    if period == 7:
        idx = dtd.dayofweek
    else:
        idx = (dtd.dayofyear % period)
    idx = np.asarray(idx, dtype=float)
    ang = 2*np.pi*idx/float(period)
    return np.cos(ang), np.sin(ang)

def zscore(a, w):
    s = pd.Series(a).astype(float)
    mu = s.rolling(w, min_periods=max(3, w//3)).mean()
    sd = s.rolling(w, min_periods=max(3, w//3)).std(ddof=0)
    return ((s - mu) / (sd.replace(0, np.nan))).fillna(0.0).values

def kpis_from_signal(returns, signal, fee_bps=5.0, slippage_bps=0.0, freq=252):
    fee = fee_bps/10000.0 + slippage_bps/10000.0
    flips = (np.abs(np.diff(np.r_[0, signal])) > 0).astype(int)
    r = (signal * returns) - flips * fee
    r = pd.Series(r).fillna(0.0)
    eq = (1 + r).cumprod()
    cagr = (1 + r).prod() ** (freq / max(len(r),1)) - 1
    vol  = r.std() * np.sqrt(freq)
    sharpe = (cagr / vol) if vol > 0 else np.nan
    maxdd = (eq / eq.cummax() - 1).min()
    return dict(Sharpe=float(sharpe), CAGR=float(cagr),
                total_return=float(eq.iloc[-1]-1), MaxDD=float(maxdd), vol_annual=float(vol))

def strat_sharpe_from_probs(p, ret_next, fee_bps=5.0, slippage_bps=0.0, thr=0.55):
    s = (p >= thr).astype(int)
    return kpis_from_signal(ret_next, s, fee_bps=fee_bps, slippage_bps=slippage_bps)["Sharpe"]


In [47]:
new_feats = []

# Volatility (rolling std of daily returns) if close present
if "close" in df.columns:
    ret1 = pd.Series(df["close"]).pct_change()
    for w in [5, 10, 20]:
        df[f"vol_std{w}"] = ret1.rolling(w, min_periods=max(3, w//3)).std(ddof=0)
        new_feats.append(f"vol_std{w}")

# Trend via simple slopes on 'close' (if present)
def slope(series, w):
    s = pd.Series(series).astype(float)
    idx = np.arange(len(s))
    # rolling slope via cov/var
    return (s.rolling(w).apply(
        lambda x: np.cov(np.arange(len(x)), x, ddof=0)[0,1]/np.var(np.arange(len(x)), ddof=0) if np.var(np.arange(len(x)), ddof=0)>0 else 0.0,
        raw=False
    ))
if "close" in df.columns:
    for w in [10, 20, 60]:
        nm = f"trend_slope{w}"
        df[nm] = slope(df["close"], w).fillna(0.0).values
        new_feats.append(nm)

# Ret1 z-scores (if ret1 present or derivable)
if "ret1" not in df.columns and "close" in df.columns:
    df["ret1"] = pd.Series(df["close"]).pct_change()
if "ret1" in df.columns:
    for w in [5, 10, 20]:
        df[f"ret1_z{w}"] = zscore(df["ret1"].fillna(0).to_numpy(), w)
        new_feats.append(f"ret1_z{w}")

# Seasonality (day-of-week)
c7, s7 = sincos_time(df["date"], period=7)
df["dow_cos"], df["dow_sin"] = c7, s7
new_feats += ["dow_cos","dow_sin"]

# Cross-interactions with market context if available
cross_feats = []
ctx = ["mkt_ret1","mkt_ret5","vix_chg1"]
base = []
for b in ["macd","macd_signal","ret1_z5","ret1_z10","ret1_z20"]:
    if b in df.columns: base.append(b)
if "ret1" in df.columns and "ret1_z5" not in base:
    base.append("ret1")
for a in ctx:
    if a in df.columns:
        for b in base:
            nm = f"{a}_x_{b.split('_')[0]}"
            df[nm] = df[a].astype(float) * df[b].astype(float)
            cross_feats.append(nm)

print("New features created:", len(new_feats)+len(cross_feats))
print("Examples:", (new_feats+cross_feats)[:12])


New features created: 26
Examples: ['vol_std5', 'vol_std10', 'vol_std20', 'trend_slope10', 'trend_slope20', 'trend_slope60', 'ret1_z5', 'ret1_z10', 'ret1_z20', 'dow_cos', 'dow_sin', 'mkt_ret1_x_macd']


In [48]:
df = df.sort_values("date").reset_index(drop=True)
n = len(df); n_tr = int(0.6*n); n_va = int(0.2*n)
df_tr = df.iloc[:n_tr].copy()
df_va = df.iloc[n_tr:n_tr+n_va].copy()
df_te = df.iloc[n_tr+n_va:].copy()

print("Re-split with engineered features:")
print("train/val/test:", df_tr.shape, df_va.shape, df_te.shape)


Re-split with engineered features:
train/val/test: (1611, 39) (537, 39) (538, 39)


In [49]:
# Base features from artifacts (if exist)
try:
    feat_art = json.loads((ART_DIR/"feature_list.json").read_text(encoding="utf-8"))
except Exception:
    feat_art = []

candidate_lists = []
for name in ["new_feats", "cross_feats"]:
    if name in globals() and isinstance(globals()[name], (list, tuple)):
        candidate_lists.extend(list(globals()[name]))

requested = list(feat_art) + list(candidate_lists)

drop_cols = {"y","ret_next","date","ticker","symbol"}
seen, all_feats, skipped = set(), [], []
for c in requested:
    if c in seen or c in drop_cols: 
        continue
    if c in df.columns and pd.api.types.is_numeric_dtype(df[c]):
        all_feats.append(c); seen.add(c)
    else:
        skipped.append(c)

if skipped:
    print("Skipping (missing/non-numeric):", skipped[:12], ("..." if len(skipped) > 12 else ""))
print("Final feature count:", len(all_feats))
print("First 10:", all_feats[:10])

# Ensure returns available for VA/TE for threshold tuning/backtest
def ensure_ret_next(d):
    if "ret_next" in d.columns:
        return d["ret_next"].astype(float).to_numpy()
    if "close" in d.columns:
        return pd.Series(d["close"]).pct_change().shift(-1).to_numpy()
    raise KeyError("Need 'ret_next' or 'close' to compute returns.")
r_va = ensure_ret_next(df_va)
r_te = ensure_ret_next(df_te)

# Arrays
X_tr = df_tr[all_feats].to_numpy(); y_tr = df_tr["y"].astype(int).to_numpy()
X_va = df_va[all_feats].to_numpy(); y_va = df_va["y"].astype(int).to_numpy()
X_te = df_te[all_feats].to_numpy(); y_te = df_te["y"].astype(int).to_numpy()

print("Shapes -> X_tr:", X_tr.shape, "X_va:", X_va.shape, "X_te:", X_te.shape)


Final feature count: 33
First 10: ['close', 'high', 'low', 'macd', 'macd_signal', 'mkt_ret1', 'mkt_ret5', 'open', 'ret1', 'ret10']
Shapes -> X_tr: (1611, 33) X_va: (537, 33) X_te: (538, 33)


In [50]:
# PATCHED Cell 6 — Impute → Scale → Sweep LR
from sklearn.impute import SimpleImputer
from sklearn.utils.class_weight import compute_class_weight

# 1) Impute on TRAIN only, then apply to VAL/TEST
imp = SimpleImputer(strategy="median")
X_tr_i = imp.fit_transform(X_tr)
X_va_i = imp.transform(X_va)
X_te_i = imp.transform(X_te)

# Replace any inf that might sneak in (just in case)
X_tr_i = np.nan_to_num(X_tr_i, posinf=0.0, neginf=0.0)
X_va_i = np.nan_to_num(X_va_i, posinf=0.0, neginf=0.0)
X_te_i = np.nan_to_num(X_te_i, posinf=0.0, neginf=0.0)

# 2) Standardize
scaler = StandardScaler().fit(X_tr_i)
X_tr_s = scaler.transform(X_tr_i)
X_va_s = scaler.transform(X_va_i)
X_te_s = scaler.transform(X_te_i)

# 3) Hyperparam sweep for LR (balanced)
LR_GRID = {"C":[0.1, 0.5, 1.0, 2.0, 5.0]}

def train_eval_lr(C=1.0, penalty="l2", class_weight="balanced"):
    lr = LogisticRegression(
        C=C, penalty=penalty, solver="lbfgs", max_iter=2000,
        class_weight=class_weight, random_state=SEED
    )
    lr.fit(X_tr_s, y_tr)
    p_va = lr.predict_proba(X_va_s)[:,1]
    p_te = lr.predict_proba(X_te_s)[:,1]
    m_va = metric_block(y_va, p_va)
    m_te = metric_block(y_te, p_te)
    return lr, p_va, p_te, m_va, m_te

rows = []; best = None
for C in LR_GRID["C"]:
    lr, pva, pte, mva, mte = train_eval_lr(C=C)
    row = {"C": C, "val_AUC": mva["AUC"], "test_AUC": mte["AUC"], "val_PR": mva["PR_AUC"], "test_PR": mte["PR_AUC"]}
    rows.append(row)
    if best is None or row["val_AUC"] > best["val_AUC"]:
        best = dict(**row, model=lr, p_va=pva, p_te=pte)

hgrid = pd.DataFrame(rows).sort_values("val_AUC", ascending=False)
hgrid.to_csv(DATA_DIR/"nb13_hparam_lr.csv", index=False)
print("Top LR configs (by val AUC):")
display(hgrid.head(10))
print("Best (val AUC):", round(best["val_AUC"], 4))
print("NaN check (should be 0s):", 
      np.isnan(X_tr_s).sum(), np.isnan(X_va_s).sum(), np.isnan(X_te_s).sum())


Top LR configs (by val AUC):


,C,val_AUC,test_AUC,val_PR,test_PR
0,0.1,0.474235,0.475888,0.491897,0.530520
1,0.5,0.472625,0.476250,0.489142,0.529797
2,1.0,0.472042,0.475162,0.488436,0.528502
4,5.0,0.472000,0.470782,0.487706,0.525420
3,2.0,0.471834,0.473907,0.488461,0.527526


Best (val AUC): 0.4742
NaN check (should be 0s): 0 0 0


In [51]:
FEE_BPS = 5.0
SLIP_BPS = 0.0

thr_grid = np.linspace(0.50, 0.65, 31)
scores = [(t, strat_sharpe_from_probs(best["p_va"], r_va, fee_bps=FEE_BPS, slippage_bps=SLIP_BPS, thr=t)) for t in thr_grid]
t_star, val_sharpe = max(scores, key=lambda x: (x[1], -abs(x[0]-0.59)))  # tie-break near 0.59

# TEST block
p_te = best["model"].predict_proba(X_te_s)[:,1]
eval_te = metric_block(y_te, p_te)
sig_te = (p_te >= t_star).astype(int)
bt_te  = kpis_from_signal(r_te, sig_te, fee_bps=FEE_BPS, slippage_bps=SLIP_BPS)

print(f"VAL Sharpe tuned threshold: {t_star:.3f} | Sharpe(val): {val_sharpe:.3f}")
test_block = {
    "AUC_test": round(eval_te["AUC"], 4),
    "Brier": round(eval_te["Brier"], 4),
    "LogLoss": round(eval_te["LogLoss"], 4),
    "Sharpe": round(bt_te["Sharpe"], 4),
    "CAGR": round(bt_te["CAGR"], 4),
    "total_return": round(bt_te["total_return"], 4),
    "MaxDD": round(bt_te["MaxDD"], 4),
    "thr": round(float(t_star), 3),
}
print("TEST block:", test_block)


VAL Sharpe tuned threshold: 0.505 | Sharpe(val): 0.288
TEST block: {'AUC_test': 0.4759, 'Brier': 0.2585, 'LogLoss': 0.7102, 'Sharpe': -0.9772, 'CAGR': -0.1607, 'total_return': -0.3121, 'MaxDD': -0.334, 'thr': 0.505}


In [52]:
# Load PROD model+scaler+feature list if available
auc_prod = None
try:
    prod_lr = json.loads((ART_DIR/"backtest_summary.json").read_text())  # also confirms artifacts exist
    lr_prod  = pd.read_pickle(ART_DIR/"lr.joblib") if (ART_DIR/"lr.joblib").suffix==".pkl" else None
except Exception:
    lr_prod = None

# Safer loader (joblib)
from joblib import load
try:
    lr_prod = load(ART_DIR/"lr.joblib")
    scaler_prod = load(ART_DIR/"scaler.joblib") if (ART_DIR/"scaler.joblib").exists() else None
    feat_prod = json.loads((ART_DIR/"feature_list.json").read_text(encoding="utf-8"))
    Xp = df_te[[c for c in feat_prod if c in df_te.columns]].to_numpy()
    if scaler_prod is not None: Xp = scaler_prod.transform(Xp)
    p_prod_te = lr_prod.predict_proba(Xp)[:,1]
    auc_prod = roc_auc_score(y_te, p_prod_te)
except Exception as e:
    print("PROD compare skipped:", e)

auc_nb13 = roc_auc_score(y_te, p_te)
if auc_prod is not None:
    print("AUC(PROD) :", round(float(auc_prod), 4))
print("AUC(NB13) :", round(float(auc_nb13), 4))
if auc_prod is not None:
    print("ΔAUC (NB13-PROD):", round(float(auc_nb13 - auc_prod), 4))


AUC(PROD) : 0.479
AUC(NB13) : 0.4759
ΔAUC (NB13-PROD): -0.0031


In [53]:
from sklearn.impute import SimpleImputer

# Define groups (then intersect with all_feats to avoid missing cols)
groups = {
    "price_core": ["open","high","low","close","volume"],
    "volatility": [c for c in all_feats if c.startswith("vol_std") or c in ["vol10","volz"]],
    "trend": [c for c in all_feats if c.startswith("trend_slope")],
    "zscore": [c for c in all_feats if c.startswith("ret1_z")],
    "seasonality": [c for c in ["dow_cos","dow_sin"] if c in all_feats],
    "ta": [c for c in ["macd","macd_signal","rsi14"] if c in all_feats],
    "market_ctx": [c for c in ["mkt_ret1","mkt_ret5","vix_chg1","spy_close","vix_close"] if c in all_feats],
    "interactions": [c for c in all_feats if "_x_" in c],
}

base_auc = roc_auc_score(y_va, best["p_va"])
abl_rows = []

for gname, cols in groups.items():
    cols = [c for c in cols if c in all_feats]
    if not cols:
        continue
    keep = [c for c in all_feats if c not in cols]
    if not keep:
        continue

    # Build reduced TRAIN/VAL matrices
    Xtr_g = df_tr[keep].to_numpy()
    Xva_g = df_va[keep].to_numpy()

    # Impute (fit on TRAIN only) → sanitize → scale (fit on TRAIN only)
    imp_g = SimpleImputer(strategy="median")
    Xtr_g = imp_g.fit_transform(Xtr_g)
    Xva_g = imp_g.transform(Xva_g)

    Xtr_g = np.nan_to_num(Xtr_g, posinf=0.0, neginf=0.0)
    Xva_g = np.nan_to_num(Xva_g, posinf=0.0, neginf=0.0)

    sc_g = StandardScaler().fit(Xtr_g)
    Xtr_g = sc_g.transform(Xtr_g)
    Xva_g = sc_g.transform(Xva_g)

    # Retrain LR with same C as the best model
    C = best["C"]
    lr_g = LogisticRegression(
        C=C, penalty="l2", solver="lbfgs", max_iter=2000,
        class_weight="balanced", random_state=SEED
    )
    lr_g.fit(Xtr_g, y_tr)
    p_va_g = lr_g.predict_proba(Xva_g)[:, 1]
    auc_g = roc_auc_score(y_va, p_va_g)

    abl_rows.append({
        "group": gname,
        "removed": len(cols),
        "val_AUC": auc_g,
        "delta": auc_g - base_auc
    })

ablt = pd.DataFrame(abl_rows).sort_values("delta").reset_index(drop=True)
print("Ablations (val AUC drop when group removed):")
display(ablt.head(10))
ablt.to_csv(DATA_DIR/"nb13_ablation_groups.csv", index=False)
print("Saved:", DATA_DIR/"nb13_ablation_groups.csv")


Ablations (val AUC drop when group removed):


,group,removed,val_AUC,delta
0,trend,3,0.463241,-0.010995
1,interactions,6,0.466600,-0.007635
2,zscore,3,0.470987,-0.003248
3,price_core,5,0.473610,-0.000625
4,seasonality,2,0.474915,0.000680
5,ta,3,0.475235,0.001000
6,volatility,5,0.476956,0.002721
7,market_ctx,3,0.491393,0.017158


Saved: data\nb13_ablation_groups.csv


In [54]:
from joblib import dump
ts = datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
dump(best["model"], ART_DIR/f"nb13_lr_{ts}.joblib")
dump(scaler, ART_DIR/f"nb13_scaler_{ts}.joblib")
with open(ART_DIR/f"nb13_threshold_{ts}.json","w",encoding="utf-8") as f:
    json.dump({"tau": float(t_star), "grid": list(map(float, thr_grid)), "val_sharpe": float(val_sharpe)}, f, indent=2)
print("Saved:", ART_DIR/f"nb13_lr_{ts}.joblib", "|", ART_DIR/f"nb13_scaler_{ts}.joblib", "|", ART_DIR/f"nb13_threshold_{ts}.json")


Saved: artifacts\nb13_lr_20251023T011130Z.joblib | artifacts\nb13_scaler_20251023T011130Z.joblib | artifacts\nb13_threshold_20251023T011130Z.json
